In [1]:
import fastai
from fastai import *          # Quick access to most common functionality
from fastai.vision import *   # Quick access to computer vision functionality
from fastai.callbacks import *
from torchvision.models import vgg16_bn, vgg16

In [2]:
PATH = Path('/DATA/kaggle/imgnetloc/ILSVRC/Data/CLS-LOC/')
PATH_TRN = PATH/'train'

sz_lr=128
scale,bs = 4,64
sz_hr = sz_lr*scale

In [3]:
classes = list(PATH_TRN.iterdir())
fnames_full = []
for class_folder in progress_bar(classes):
    for fname in class_folder.iterdir():
        fnames_full.append(fname)

np.random.seed(42)
keep_pct = 0.01
keeps = np.random.rand(len(fnames_full)) < keep_pct
image_fns = np.array(fnames_full, copy=False)[keeps]
len(image_fns)

12829

In [4]:
valid_pct = 0.1
src = (ImageToImageList(image_fns)
       .random_split_by_pct(valid_pct)
       .label_from_func(lambda x: x))

In [5]:

def get_data(bs, sz_lr, sz_hr):
    data = (src
            .transform(get_transforms(), size=sz_lr)
            .transform_labels(size=sz_hr)
            .databunch(bs=bs)
            .normalize(imagenet_stats, do_y=True))
    return data

bs = 8
sz_hr = 224
sz_lr = sz_hr // 4
data = get_data(bs, sz_lr, sz_hr)

In [6]:
x,y = data.dl().one_batch()
x.shape, y.shape

(torch.Size([12, 3, 56, 56]), torch.Size([12, 3, 224, 224]))

#Model

In [7]:
def conv(ni, nf, kernel_size=3, actn=True):
    layers = [nn.Conv2d(ni, nf, kernel_size, padding=kernel_size//2)]
    if actn: layers.append(nn.ReLU(True))
    return nn.Sequential(*layers)

In [8]:
class ResSequential(nn.Module):
    def __init__(self, layers, res_scale=1.0):
        super().__init__()
        self.res_scale = res_scale
        self.m = nn.Sequential(*layers)

    def forward(self, x):
        x = x + self.m(x) * self.res_scale
        return x

In [9]:
def res_block(nf):
    return ResSequential(
        [conv(nf, nf), conv(nf, nf, actn=False)],
        0.1)

In [10]:
def upsample(ni, nf, scale):
    layers = []
    for i in range(int(math.log(scale,2))):
        layers += [conv(ni, nf*4), nn.PixelShuffle(2)]
    return nn.Sequential(*layers)


In [11]:
class SrResnet(nn.Module):
    def __init__(self, nf, scale):
        super().__init__()
        features = [conv(3, 64)]
        for i in range(8): features.append(res_block(64))
        features += [conv(64,64), upsample(64, 64, scale),
                     nn.BatchNorm2d(64),
                     conv(64, 3, actn=False)]
        self.features = nn.Sequential(*features)
    
    def forward(self, x): return self.features(x)

In [12]:
model = SrResnet(64, scale)

In [13]:
def icnr(x, scale, init=nn.init.kaiming_normal_):
    new_shape = [int(x.shape[0] / (scale ** 2))] + list(x.shape[1:])
    subkernel = torch.zeros(new_shape)
    subkernel = init(subkernel)
    subkernel = subkernel.transpose(0, 1)
    subkernel = subkernel.contiguous().view(subkernel.shape[0],
                                            subkernel.shape[1], -1)
    kernel = subkernel.repeat(1, 1, scale ** 2)
    transposed_shape = [x.shape[1]] + [x.shape[0]] + list(x.shape[2:])
    kernel = kernel.contiguous().view(transposed_shape)
    kernel = kernel.transpose(0, 1)
    return kernel

In [ ]:
learn = Learner(data, model, loss_func=F.mse_loss)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 1e-3
learn.fit_one_cycle(1, lr)

In [ ]:
learn.save('enhance')

In [ ]:
learn = learn.load('enhance')

In [14]:
def plot_x_y_pred(x, pred, y, figsize):
    rows=x.shape[0]
    fig, axs = plt.subplots(rows,3,figsize=figsize)
    for i in range(rows):
        show_image(x[i], ax=axs[i, 0])
        show_image(pred[i], ax=axs[i, 1])
        show_image(y[i], ax=axs[i, 2])
    plt.tight_layout()    
    

In [ ]:
x, y = next(iter(learn.data.valid_dl))
y_pred = model(x)
x.shape, y.shape, y_pred.shape
x_norm = learn.data.denorm(x.detach())
y_norm = learn.data.denorm(y.detach())
y_pred_norm = learn.data.denorm(y_pred.detach())
plot_x_y_pred(x_norm.cpu()[0:3], y_pred_norm.cpu()[0:3], y_norm.cpu()[0:3], figsize=y_pred_norm.shape[-2:])

In [15]:
m_vgg_feat = vgg16(True).cuda().eval().features
blocks = [i-1 for i,o in enumerate(children(m_vgg_feat))
              if isinstance(o,nn.MaxPool2d)]
blocks, [m_vgg_feat[i] for i in blocks]

([3, 8, 15, 22, 29],
 [ReLU(inplace), ReLU(inplace), ReLU(inplace), ReLU(inplace), ReLU(inplace)])

In [16]:
class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts

    def make_feature(self, bs, o, clone=False):
        feat = o.view(bs, -1)
        if clone: feat = feat.clone()
        return feat
    
    def make_features(self, x, clone=False):
        bs = x.shape[0]
        self.m_feat(x)
        return [self.make_feature(bs, o, clone) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [F.l1_loss(f_in, f_out)*w
                            for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.loss = sum(self.feat_losses)
        return self.loss
        
class ReportFeatLoss(LearnerCallback):
    _order = -20 #Needs to run before the recorder
    
    def __
    def on_train_begin(self, **kwargs):
        self.learn.recorder.add_metric_names(['loss1', 'loss2'])
    
    def on_epoch_begin(self, **kwargs):
        self.avg1, self.avg2, self.nums = 0., 0., 0
    
    def on_batch_end(self, last_target, train, **kwargs):
        if not train:
            bs = last_target.size(0)
            self.avg1 += bs * learn.loss_fn.loss1.detach()
            self.avg2 += bs * learn.loss_fn.loss2.detach()
            self.nums += bs
    
    def on_epoch_end(self, **kwargs):
        self.learn.recorder.add_metrics([self.avg1/self.nums, self.avg2/self.nums])


In [22]:
LearnerCallback??

In [17]:
conv_shuffle = model.features[10][0][0]
kernel = icnr(conv_shuffle.weight, scale=scale)
conv_shuffle.weight.data.copy_(kernel);

conv_shuffle = model.features[10][2][0]
kernel = icnr(conv_shuffle.weight, scale=scale)
conv_shuffle.weight.data.copy_(kernel);

In [18]:
bs = 32
sz_hr = 224
sz_lr = sz_hr // 4
data = get_data(bs, sz_lr, sz_hr)

feat_loss = FeatureLoss(m_vgg_feat, blocks[:2], [0.26,0.74])
learn = Learner(data, nn.DataParallel(model), loss_func=feat_loss)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [19]:
lr = 1e-3
learn.fit_one_cycle(1, lr)

Total time: 04:39
epoch  train_loss  valid_loss
1      0.831466    0.834908    (04:39)



In [20]:
learn.save('enhance_feat')

In [21]:
learn = learn.load('enhance_feat')

In [23]:
x, y = next(iter(learn.data.valid_dl))
y_pred = model(x)
x.shape, y.shape, y_pred.shape
x_norm = learn.data.denorm(x.detach().cpu())
y_norm = learn.data.denorm(y.detach().cpu())
y_pred_norm = learn.data.denorm(y_pred.detach().cpu())
plot_x_y_pred(x_norm[0:3], y_pred_norm[0:3], y_norm[0:3], figsize=y_pred_norm.shape[-2:])
